<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/InraTour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import itertools
import math
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [3]:
home = ['Pleione', (-77 , -146.78125 , -344.125)]

In [4]:
points = [    ['12 Trianguli',  (-77.15625 , -76.8125 , -120.6875)],
              ['HIP 12099',     (-101.90625 , -95.46875 , -165.59375)],
              ['HIP 16824',     (-49.21875 , -89.375 , -187.1875)],
              ['HIP 15329',     (-87.84375 , -52 , -167.375)],
              ['Alnath',        (-3.875 , -7.375 , -133.65625)],
              ['HIP 7158',      (-3.78125 , -143.84375 , -33.8125)],
              ['HIP 59382',     (19.03125 , 130.09375 , -11.59375)],
              ['Conn',          (-36.59375 , -9.1875 , -20.5)],
              ['Hermitage',     (-28.75 , 25 , 10.4375)],
              ['LP 389-95',     (98.84375 , 7 , -7.71875)],
          ]

In [30]:
sequence = [4, 9, 6, 8, 7, 5, 0, 3, 1, 2] #list(range( 0,len(points)))

In [39]:

def metric_dist( pt1, pt2):
  d = 0.0
  d += (pt1[0] - pt2[0]) **2
  d += (pt1[1] - pt2[1]) **2
  d += (pt1[2] - pt2[2]) **2
  d = math.sqrt(d)
  return d

def dist( pt1, pt2):
  d = metric_dist(pt1, pt2)
  return d #math.ceil(d/74)

def score(sequence):
  d = 0.0
  slen = len(sequence)
  d+=  dist( home[1], points[sequence[0]][1])
  for i in range(1, slen):
    d+= dist( points[sequence[i]][1], points[sequence[i-1]][1])

  d+= dist( home[1], points[sequence[slen-1]][1])
  return d

def search_for_best():
  i = 0
  magnitude = 2

  best = sequence
  bestScore = score(sequence)
  for cur in itertools.permutations(sequence):
   if i % 10 **magnitude == 0:
    if i % 10 **(magnitude+2) == 0:
      print(".")
    else:
      print(".", end='')

    sco = score(cur)
    if sco< bestScore:
      best = cur
      bestScore = sco
   i = i + 1

  print("")
  print(str(bestScore))
  print(best)

1334.5975515058062
(4, 9, 6, 8, 7, 5, 0, 3, 1, 2)

In [7]:
def nicePrint( seq ):
  print("| System        | Distance |")
  print("|---------------|----------|")

  d = dist( home[1], points[seq[0]][1])
  print(f"|{home[0]:15.15s}| {d:.0f}|")
  for i in range(0, len(seq)-1):
    d= dist( points[seq[i]][1], points[seq[i+1]][1])
    print(f"|{points[seq[i]][0]:15.15s}| {d:.0f}|")
  
  d= dist( home[1], points[seq[-1]][1])
  print(f"|{points[seq[-1]][0]:15.15s}| {d:.0f}|")
  print(f"|{home[0]:15.15s}| -|")


In [8]:
nicePrint( (4, 9, 6, 8, 7, 5, 0, 3, 1, 2) )

| System        | Distance |
|---------------|----------|
|Pleione        | 263|
|Alnath         | 163|
|LP 389-95      | 147|
|HIP 59382      | 118|
|Hermitage      | 47|
|Conn           | 139|
|HIP 7158       | 132|
|12 Trianguli   | 54|
|HIP 15329      | 46|
|HIP 12099      | 57|
|HIP 16824      | 169|
|Pleione        | -|


In [9]:
def makeFrame( seq ):
  names = []
  xs = []
  ys = []
  zs = []

  names.append( home[0])
  xs.append( home[1][0])
  ys.append( home[1][1])
  zs.append( home[1][2])


  for i in range(0, len(seq)):
    thePoint = points[seq[i]]
    names.append( thePoint[0])
    xs.append( thePoint[1][0])
    ys.append( thePoint[1][1])
    zs.append( thePoint[1][2])

  names.append( home[0])
  xs.append( home[1][0])
  ys.append( home[1][1])
  zs.append( home[1][2])

  data = { 'names': names, 'xs':xs, 'ys':ys, 'zs':zs }
  return pd.DataFrame( data)

In [27]:
def getScene():
        return dict(
            xaxis=dict(
                backgroundcolor="rgb(0,0,0)",
                gridcolor="grey",
                showbackground=False,
                zerolinecolor="white", ),
            yaxis=dict(
                backgroundcolor="rgb(0,0,0)",
                gridcolor="grey",
                showbackground=False,
                zerolinecolor="white", ),
            zaxis=dict(
                backgroundcolor="rgb(0,0,0)",
                gridcolor="grey",
                showbackground=False,
                zerolinecolor="white", ),
            aspectratio=dict(x=1, y=1, z=0.7),
            aspectmode="manual"
)
        
def getLayout(theTitle):
        return go.Layout(title=theTitle,
                         scene=getScene(),
                         width=500,
                         height=350,
                         autosize=False,
                         paper_bgcolor='rgb(0,0,0)',
                         plot_bgcolor='rgb(0,0,0)',
                         clickmode='event+select',
                         font=dict(
                             family="Courier New, monospace",
                             size=12,
                             color="#ffffff"),
                         margin=dict(t=25, b=0, l=0, r=0),
)

In [10]:
df = makeFrame( (4, 9, 6, 8, 7, 5, 0, 3, 1, 2) )

In [28]:


fig = go.Figure(data=[go.Scatter3d(
    x=df['xs'],
    y=df['ys'],
    z=df['zs'],
    text=df['names'],
    mode='markers+lines+text'
    )],
    #showgrid=False
    )

# tight layout
fig.update_layout( getLayout("INRA Route") )
#margin=dict(l=0, r=0, b=0, t=0),
#                  paper_bgcolor='rgba(255,255,255,255)',
#                  plot_bgcolor='rgba(255,255,255,255)')
fig.show()

In [ ]:
math.factorial(10)

In [38]:
#search_for_best()

.
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
.........................................................................................